# Importing libraries and setting up connection

In [1]:
from pymongo import MongoClient
import pandas as pd
import time
import numpy as np
import re

In [2]:
client = MongoClient("localhost:27017")

In [3]:
client.list_database_names()

['Ironhack', 'admin', 'config', 'local']

In [4]:
db = client["Ironhack"]

In [5]:
c = db.get_collection("companies")

In [6]:
c

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Ironhack'), 'companies')

# Queryng

## 3 top cities

###### Three cities are selected as candidates based on the location of the video game companies that have raised the most money.

In [7]:
#c.distinct("category_code")

In [8]:
query = {"category_code": "games_video",
         "total_money_raised": {"$regex": re.compile(r"^(€|\$|¥)[1-9]\d*M$")},
         "offices.city": {"$exists": True, "$ne": None}}
projection = {"name": 1, "_id" : 0, "total_money_raised": 1, "offices.city": 1}
results = list(c.find(query, projection).sort("total_money_raised", -1))

# Dataframe
df = pd.DataFrame(results)

# Create new columns for "city" and cleaning the dataframe
df['offices'] = df['offices'].apply(lambda x: x[0]['city'] if x else None)
df['offices'].replace('', np.nan, inplace=True)
df.dropna(subset=['offices'], inplace=True)
pd.set_option('display.max_rows', None)

df


,name,total_money_raised,offices
0,NYX Interactive,€5M,Stockholm
1,NYX Interactive,€5M,Stockholm
2,Selatra,€4M,Cork
3,Bigpoint,€420M,Hamburg
4,Owlient,€3M,Paris
5,sevenload,€25M,Cologne
6,GREE,¥464M,Tokyo
7,Revision3,$9M,San Francisco
8,PermissionTV,$9M,Waltham
9,WorldWide Biggies,$9M,New York


In [9]:
df["offices"].value_counts().head(3)

offices
San Francisco    15
New York         13
Santa Monica      5
Name: count, dtype: int64

## Tech companies

###### SAN FRANCISCO

In [10]:
cond_1 = {"$or": [
        {"category_code": "biotech"},
        {"category_code": "cleantech"},
        {"category_code": "games_video"},
        {"category_code": "nanotech"},
        {"category_code": "software"},
        {"category_code": "web"}
    ]}
cond_2 = {"funding_rounds.raised_amount": {"$gte": 1000000}}
cond_3 = {"number_of_employees": {"$lte": 100}}
cond_4 = {"offices.city": "San Francisco"}


query = {"$and": [cond_1, cond_2, cond_3, cond_4]}
projection = {"name": 1, "_id": 0, "offices.city":1, "offices.latitude": 1, "offices.longitude": 1}
tech_startups_sf = list(c.find(query, projection))

df_sf = pd.DataFrame(tech_startups_sf)


In [11]:
# Create new columns for "city", "latitude", and "longitude"
df_sf['city'] = df_sf['offices'].apply(lambda x: x[0]['city'] if x else None)
df_sf['latitude'] = df_sf['offices'].apply(lambda x: x[0]['latitude'] if x else None)
df_sf['longitude'] = df_sf['offices'].apply(lambda x: x[0]['longitude'] if x else None)

# Drop columnes, nan values and wrong cities
df_sf.drop('offices', axis=1, inplace=True)
df_sf.dropna(subset=['latitude'], inplace=True)
df_sf = df_sf.loc[df_sf['city'].str.contains('San Francisco')]
df_sf

,name,city,latitude,longitude
0,Kyte,San Francisco,37.788482,-122.409173
1,CastTV,San Francisco,37.780716,-122.393913
2,Pageflakes,San Francisco,37.758113,-122.414689
4,Bebo,San Francisco,37.782103,-122.401116
5,Seesmic,San Francisco,37.775196,-122.419204
6,GoingOn,San Francisco,37.782263,-122.392142
7,Flixster,San Francisco,37.766909,-122.406676
8,hi5,San Francisco,37.788668,-122.400558
9,Curse,San Francisco,37.787092,-122.399972
10,Yola,San Francisco,37.791115,-122.395475


In [12]:
#df_sf.to_csv('../data/tech_companies_sf.csv', index=False)

###### NEW YORK

In [13]:
query = {"$or": [
        {"category_code": "design"},
        {"category_code": "biotech"},
        {"category_code": "cleantech"},
        {"category_code": "games_video"},
        {"category_code": "nanotech"},
        {"category_code": "software"},
        {"category_code": "web"}
    ]}
cond_2 = {"funding_rounds.raised_amount": {"$gte": 1000000}}
cond_3 = {"number_of_employees": {"$lte": 100}}
cond_4 = {"offices.city": "New York"}


query = {"$and": [query, cond_2, cond_3, cond_4]}
projection = {"name": 1, "_id": 0, "offices.city":1, "offices.latitude": 1, "offices.longitude": 1}
tech_startups_ny = list(c.find(query, projection))

df_ny = pd.DataFrame(tech_startups_ny)
df_ny

,name,offices
0,Wetpaint,"[{'city': 'Seattle', 'latitude': 47.603122, 'l..."
1,Joost,"[{'city': 'New York', 'latitude': 40.7464969, ..."
2,Meetup,"[{'city': 'New York', 'latitude': 40.72604, 'l..."
3,Pando Networks,"[{'city': 'New York', 'latitude': 40.722655, '..."
4,SpiralFrog,"[{'city': 'New York', 'latitude': 37.09024, 'l..."
5,Tutor,"[{'city': 'New York', 'latitude': 40.707921, '..."
6,OMGPOP,"[{'city': 'New York', 'latitude': 40.723384, '..."
7,Behance,"[{'city': 'New York', 'latitude': 40.74406, 'l..."
8,in2apps,"[{'city': 'New York', 'latitude': None, 'longi..."
9,WEbook,"[{'city': 'New York', 'latitude': 38.984172, '..."


In [14]:
# Create new columns for "city", "latitude", and "longitude"
df_ny['city'] = df_ny['offices'].apply(lambda x: x[0]['city'] if x else None)
df_ny['latitude'] = df_ny['offices'].apply(lambda x: x[0]['latitude'] if x else None)
df_ny['longitude'] = df_ny['offices'].apply(lambda x: x[0]['longitude'] if x else None)

# Drop columnes, nan values and wrong cities
df_ny.drop('offices', axis=1, inplace=True)
df_ny.dropna(subset=['latitude'], inplace=True)
df_ny = df_ny.loc[df_ny['city'].str.contains('New York')]
df_ny

,name,city,latitude,longitude
1,Joost,New York,40.746497,-74.009447
2,Meetup,New York,40.726040,-73.995722
3,Pando Networks,New York,40.722655,-73.998730
4,SpiralFrog,New York,37.090240,-95.712891
5,Tutor,New York,40.707921,-74.004792
6,OMGPOP,New York,40.723384,-74.001704
7,Behance,New York,40.744060,-74.004592
9,WEbook,New York,38.984172,-77.095965
10,WeShow,New York,40.752516,-73.973072
11,SimulScribe,New York,40.762673,-73.969296


In [15]:
#df_ny.to_csv('../data/tech_companies_ny.csv', index=False)

###### NEW YORK

In [16]:
query = {"$or": [
        {"category_code": "biotech"},
        {"category_code": "cleantech"},
        {"category_code": "games_video"},
        {"category_code": "nanotech"},
        {"category_code": "software"},
        {"category_code": "web"}
    ]}
cond_2 = {"funding_rounds.raised_amount": {"$gte": 1000000}}
cond_3 = {"number_of_employees": {"$lte": 100}}
cond_4 = {"offices.city": "Santa Monica"}


query = {"$and": [query, cond_2, cond_3, cond_4]}
projection = {"name": 1, "_id": 0, "offices.city":1, "offices.latitude": 1, "offices.longitude": 1}
tech_startups_sm = list(c.find(query, projection))


df_sm = pd.DataFrame(tech_startups_sm)
df_sm

,name,offices
0,Docstoc,"[{'city': 'Santa Monica', 'latitude': 34.01827..."
1,Goodreads,"[{'city': 'Santa Monica', 'latitude': 34.01052..."
2,PluggedIn,"[{'city': 'Santa Monica', 'latitude': 34.00796..."
3,Titan Gaming,"[{'city': 'Santa Monica', 'latitude': 53.54471..."
4,Total Beauty Media,"[{'city': 'Santa Monica', 'latitude': None, 'l..."
5,ZillionTV,"[{'city': 'Sunnyvale', 'latitude': 37.3738725,..."
6,Tongal,"[{'city': 'Santa Monica', 'latitude': 34.00711..."
7,Blip,"[{'city': 'New York', 'latitude': None, 'longi..."


In [17]:
# Create new columns for "city", "latitude", and "longitude"
df_sm['city'] = df_sm['offices'].apply(lambda x: x[0]['city'] if x else None)
df_sm['latitude'] = df_sm['offices'].apply(lambda x: x[0]['latitude'] if x else None)
df_sm['longitude'] = df_sm['offices'].apply(lambda x: x[0]['longitude'] if x else None)

# Drop columnes, nan values and wrong cities
df_sm.drop('offices', axis=1, inplace=True)
df_sm.dropna(subset=['latitude'], inplace=True)
df_sm = df_sm.loc[df_sm['city'].str.contains('Santa Monica')]
df_sm

,name,city,latitude,longitude
0,Docstoc,Santa Monica,34.018275,-118.495025
1,Goodreads,Santa Monica,34.010520,-118.495981
2,PluggedIn,Santa Monica,34.007967,-118.489681
3,Titan Gaming,Santa Monica,53.544711,-113.515769
6,Tongal,Santa Monica,34.007112,-118.489748


In [18]:
#df_sm.to_csv('../data/tech_companies_sm.csv', index=False)